In [3]:
import tensorflow as tf
x = tf.ones(shape=(2,1)) # 2 x 1'lik birler matrisi
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)


In [4]:
x = tf.zeros(shape=(2,1)) # 2 x 1'lik sıfırlar matrisi
print(x)

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


In [12]:
x = tf.random.normal(shape=(3,1), mean=0., stddev=1.) # ortalaması 0,
# standart sapması 1 olan 3x1 boyutunda rastgele bir tensor (normal distribution)
print(x)

tf.Tensor(
[[-0.31075934]
 [-0.0623241 ]
 [ 0.84930634]], shape=(3, 1), dtype=float32)


In [13]:
x = tf.random.uniform(shape=(3,1), minval=0., maxval=1.)
# 0 ile 1 arasında rastgele bir tensor (uniform distribution)
print(x)

tf.Tensor(
[[0.3207345 ]
 [0.40132582]
 [0.6137203 ]], shape=(3, 1), dtype=float32)


Numpy array'leri değiştirilebilirken tensorflow tensorları değiştirilemezdir.


In [15]:
import numpy as np
x = np.ones(shape=(2,2))
print(x)
x[0,0] = 0.
print(x)

[[1. 1.]
 [1. 1.]]
[[0. 1.]
 [1. 1.]]


In [17]:
x = tf.ones(shape=(2,2))
x[0,0] = 0

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

Fakat örneğin modelimizde ağırlıklar (w) gibi değiştirmemiz gereken array'ler
olacak. Bu durumda yapacağımız şey tensorflow'da değişken (Variable) olarak
atamak. Bu o array'in daha sonra değiştirilebileceği anlamına gelir.

In [18]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3,1)))
#initial value: başlangıc değerimiz
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.3990925 ],
       [-0.43444774],
       [-0.529121  ]], dtype=float32)>


In [19]:
# bu Variable'lar assign ile değiştirilebilir
v.assign(tf.ones((3,1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>


In [21]:
# direkt olarak değişkenin tamamını değiştirmek yerine belirli indeksi değiştirme
v[0,0].assign(3.)

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

In [22]:
# assign_add() +=
# assign_sub() -=

v.assign_add(tf.ones((3,1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [2.],
       [2.]], dtype=float32)>

In [23]:
v.assign_sub(tf.ones((3,1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

# Tensorflow'da matematiksel işlemler: tensor operasyonları

In [25]:
a = tf.ones((2,2)) #2x2 birler matrisi
print(a)
b = tf.square(a) # her bir elemanın karesi
print(b)
c = tf.sqrt(a) # her bir elemanın karekökü
print(c)
d = b + c
print(d)
e = tf.matmul(a,b) # matris çarpımı
print(e)
e *= d
print(e)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[4. 4.]
 [4. 4.]], shape=(2, 2), dtype=float32)


# gradienttape

In [32]:
input_var = tf.Variable(initial_value=3.) # 3 değeri ile başladık
# sonuna nokta koymamızın sebebi veri tipi olarak float istememiz
with tf.GradientTape() as tape:
    result = tf.square(input_var) # 3'ün karesini aldık = 9
gradient = tape.gradient(result, input_var) # 9 ile 3 arasındaki fark = 6
print(gradient)

tf.Tensor(6.0, shape=(), dtype=float32)


In [31]:
# variable gibi değişebilen bir yapı ile değil de constant gibi değişmez
# bir yapı ile çalışalım

input_const = tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(input_const)
    result = tf.square(input_const)
gradient = tape.gradient(result, input_const)
print(gradient)

# yine aynı sonucu veriyor
# fakat constant yapılarda tape.watch() ile izlemek gerekir.
# her veriyi tek tek izlemek yerine, bizim söylediğimizi izledi.

tf.Tensor(6.0, shape=(), dtype=float32)


In [36]:
# ikinci dereceden türev

time = tf.Variable(0.)
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as tape:
        position = 4.9 * time ** 2
    speed = tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)

print(speed)
print(acceleration)

# burada olan şey şu yolun formülü 4.9 * t ^ 2
# bunun türevi bize hızı verecek = 9.8 * t | t = 0 alırsak da hız = 0
# hızın türevi ivmeyi verir = 9.8 | t'ye bağlı olmadığı, sabit olduğu için 9.8



tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(9.8, shape=(), dtype=float32)


# lineer sınıflandırıcı yazmak